<b>1ο Βήμα: Εισαγωγή Βιβλιοθηκών</b>

Σε αυτό το βήμα, εισάγουμε τις απαραίτητες βιβλιοθήκες για web scraping,
επεξεργασία φυσικής γλώσσας (NLP) και διαχείριση δεδομένων. Επιπλέον, κατεβάζουμε τους απαραίτητους πόρους από το NLTK.

In [3]:
import wikipedia
import requests
import nltk
import string
import json
import math
import numpy as np
from bs4 import BeautifulSoup
from nltk import FreqDist
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import average_precision_score

# Λήφη απαραίτητων πόρων
wikipedia.set_lang("en")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jimdr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jimdr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jimdr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<b>2ο Βήμα: Ορισμός URL</b>

Αποθηκεύουμε σε μια λίστα τα URL της Wikipedia που περιέχουν τα άρθρα που θέλουμε να
επεξεργαστούμε. Τα URL αυτά αντιστοιχούν σε διάφορα θέματα σχετικά με φαγητά και
συνταγές.

In [4]:
# Λίστα URL των άρθρων από τη Wikipedia
urls = [
    "https://en.wikipedia.org/wiki/Tiropita",
    "https://en.wikipedia.org/wiki/Peinirli", 
    "https://en.wikipedia.org/wiki/Pizza",
    "https://en.wikipedia.org/wiki/Spaghetti",
    "https://en.wikipedia.org/wiki/Bread",
    "https://en.wikipedia.org/wiki/Moussaka",
    "https://en.wikipedia.org/wiki/Pastitsio",
    "https://en.wikipedia.org/wiki/Soup",
    "https://en.wikipedia.org/wiki/Galaktoboureko",
    "https://en.wikipedia.org/wiki/Bougatsa",
    "https://en.wikipedia.org/wiki/Savory_spinach_pie",
    "https://en.wikipedia.org/wiki/Gigantes_plaki",
    "https://en.wikipedia.org/wiki/Fasolada",
    "https://en.wikipedia.org/wiki/Lokma",
    "https://en.wikipedia.org/wiki/Snails_as_food",
    "https://en.wikipedia.org/wiki/Pumpkin_pie",
    "https://en.wikipedia.org/wiki/Souvlaki",
    "https://en.wikipedia.org/wiki/Doughnut",
    "https://en.wikipedia.org/wiki/Fried_egg",
    "https://en.wikipedia.org/wiki/Omelette",
    "https://en.wikipedia.org/wiki/Strapatsada",
    "https://en.wikipedia.org/wiki/Scrambled_eggs",
    "https://en.wikipedia.org/wiki/Poached_egg",
    "https://en.wikipedia.org/wiki/Milk_pie",
    "https://en.wikipedia.org/wiki/Melktert",
    "https://en.wikipedia.org/wiki/Fit-fit",
    "https://en.wikipedia.org/wiki/Cachupa",
    "https://en.wikipedia.org/wiki/Kebab",
    "https://en.wikipedia.org/wiki/Koshary",
    "https://en.wikipedia.org/wiki/Mombar",
    "https://en.wikipedia.org/wiki/Hamburger",
    "https://en.wikipedia.org/wiki/Hot_dog"
]

<b>3ο Βήμα: Ανάκτηση και Επεξεργασία Άρθρων</b>

Σε αυτό το βήμα, ανακτουμε το περιεχόμενο των άρθρων από τη Wikipedia.
Χρησιμοποιούμε τη βιβλιοθήκη BeautifulSoup για την
ανάλυση του HTML περιεχομένου, αφαιρούμε τους συνδέσμους και τις έντονες
γραμματοσειρές και εξάγουμε το κυρίως κείμενο από κάθε άρθρο σε ένα αρχείο JSON.

In [5]:
# Αρχικοποίηση λίστας για αποθήκευση άρθρων
articles = []

# Ανάκτηση άρθρων από κάθε URL
for url in urls:
    response = requests.get(url) # Κατεβάζουμε το περιεχόμενο του URL
    response.encoding = 'utf-8' # Ρύθμιση encoding
    soup = BeautifulSoup(response.text, 'html.parser') # Ανάλυση του HTML

    # Αφαίρεση ετικετών HTML όπως <a>, <b>, <strong>
    for a in soup.find_all('a'):
        a.unwrap()
    for bold in soup.find_all(['b', 'strong']):
        bold.unwrap()

    # Εξαγωγή κειμένου από παραγράφους
    paragraphs = soup.find_all('p')
    parsed_paragraphs = [p.get_text(separator=" ", strip=True) for p in paragraphs]
    articles.append(" ".join(parsed_paragraphs))

# Αποθήκευση άρθρων σε αρχείο JSON
output_filename = "articles.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

print("Articles fetched and saved successfully!")

Articles fetched and saved successfully!


<b>4ο Βήμα: Δημιουργία Αντεστραμμένου Ευρετηρίου</b>

Σε αυτό το βήμα φορτώνουμε τα επεξεργασμένα άρθρα και δημιουργούμε το αντεστραμμένο ευρετήριο,
το οποίο συνδέει κάθε όρο με τη λίστα των εγγράφων στα οποία εμφανίζεται.

In [6]:
# Φόρτωση άρθρων από το αρχείο
with open(output_filename, 'r', encoding='utf-8') as f:
    articles = json.load(f)

# Αρχικοποίηση εργαλείων του NLTK
lemmatizer = nltk.WordNetLemmatizer()
stopwords = set(nltk.corpus.stopwords.words('english'))

# Δημιουργία αντεστραμμένου ευρετηρίου 
inverted_index = {}
for doc_id, document in enumerate(articles):
    tokens = nltk.word_tokenize(document.lower()) # Διαχωρισμός σε λέξεις (tokens)
    for token in tokens:
        token = lemmatizer.lemmatize(token) # lemmatization
        if token not in stopwords and token not in string.punctuation: # Φιλτράρισμα λέξεων
            if token not in inverted_index:
                inverted_index[token] = []     
            if doc_id not in inverted_index[token]:    
                inverted_index[token].append(doc_id)

# Αποθήκευση του αντεστραμμένου ευρετηρίου
inverted_index_filename = "inverted_index.json"
with open(inverted_index_filename, 'w', encoding='utf-8') as json_file:
    json.dump(inverted_index, json_file, indent=4)

print("Inverted index created and saved successfully!")


Inverted index created and saved successfully!


<b>5ο Βήμα: Επεξεργασία Ερωτήματος Χρήστη</b>

Σε αυτό το βήμα, επεξεργαζόμαστε την ερώτηση που κάνει ο χρήστης.
Η ερώτηση χωρίζεται σε tokens,
τα κεφαλαία γράμματα μετατρέπονται σε πεζά, γίνεται lemmatization,
αφαιρούνται τα stopwords και τα σημεία στίξης.

In [7]:
# Εισαγωγή ερώτηματος χρήστη
query= input("type a query: ")

# Επεξεργασία του ερωτήματος
query_tokens=nltk.word_tokenize(query.lower())
query_tokens = [lemmatizer.lemmatize(token) for token in query_tokens if token not in stopwords and token not in string.punctuation]

print("Processed query tokens:", query_tokens)

Processed query tokens: ['cheese']


<b>6ο Βήμα: Εκτέλεση Αναζητήσεων AND/OR/NOT </b>

Σε αυτό το βήμα, εκτελούνται τρεις τύποι αναζητήσεων με βάση την ερώτηση:
1. Αναζήτηση AND, όπου επιστρέφονται τα έγγραφα που περιέχουν όλους τους όρους της ερώτησης.
2. Αναζήτηση OR, όπου επιστρέφονται τα έγγραφα που περιέχουν τουλάχιστον έναν από τους όρους της ερώτησης.
3. Αναζήτηση NOT, όπου επιστρέφονται τα έγγραφα που δεν περιέχουν κανέναν από τους όρους της ερώτησης.

In [8]:
and_docs = []
or_docs = []
not_docs = []

for doc_id, document in enumerate(articles):
    count=0
    for token in query_tokens:
        if token in inverted_index:
            if doc_id in inverted_index[token]:
                count=count+1        
    if count==len(query_tokens):
        and_docs.append(doc_id) # Έγγραφα που περιέχουν όλους τους όρους
    elif count>0:
        or_docs.append(doc_id) # Έγγραφα που περιέχουν μερικούς από τους όρους
    else:
        not_docs.append(doc_id) # Έγγραφα που δεν περιέχουν κανέναν από τους όρους

j=1

for i in and_docs:
    print(j, ". ", articles[i][0:20], "...")
    j=j+1
    
for i in or_docs:
    print(j, ". ", articles[i][0:20], "...")
    j=j+1
        
for i in not_docs:
    print(j, ". ", articles[i][0:20], "...")
    j=j+1
    
print("\n\n\n\n")

print("AND docs:", and_docs)
print("OR docs:", or_docs)
print("NOT docs:", not_docs)


1 .  Tiropita or tyropita ...
2 .  The peinirli (or pen ...
3 .   Pizza [ a ] [ 1 ] i ...
4 .   Spaghetti ( Italian ...
5 .   Bread is a staple f ...
6 .   Moussaka ( / m uː ˈ ...
7 .  Pastitsio ( Greek :  ...
8 .    Soup is a primaril ...
9 .   Bougatsa ( Greek :  ...
10 .  Savory spinach pie i ...
11 .  Gigantes plaki ( Gre ...
12 .  Lokma , is a dessert ...
13 .   A doughnut (sometim ...
14 .   A fried egg is a co ...
15 .  An omelette (sometim ...
16 .  Strapatsada (Greek:  ...
17 .   Scrambled eggs is a ...
18 .    Kebab ( UK : / k ɪ ...
19 .   A hamburger , or si ...
20 .    A hot dog [ 1 ] [  ...
21 .  Galaktoboureko ( Gre ...
22 .  Fasolada ( Greek : φ ...
23 .  Snails are eaten by  ...
24 .  Pumpkin pie is a des ...
25 .  Souvlaki ( Greek : σ ...
26 .   A poached egg is an ...
27 .  Pie susu ( Indonesia ...
28 .   Melktert ( / ˈ m ɛ  ...
29 .  Fit-fit or fir-fir ( ...
30 .  Cachupa ( Portuguese ...
31 .  Koshary , kushari or ...
32 .   Mombar (in ِ Egypti ...





AND docs: [0

<b>7ο Βήμα: Υπολογισμός TF-IDF </b>

Σε αυτό το βήμα, υπολογίζονται τα scores του TF-IDF για τους όρους
της ερώτησης σε κάθε έγγραφο. Με βάση τα scores ταξινομούνται τα έγγραφα.
Ο υπολογισμός του TF-IDF, αξιολογεί την σημασία κάθε όρου του ερωτήματος χρήστη σε ένα έγγραφο. 
Το TF αποτελεί τη συχνότητα ενός όρου σε ένα έγγραφο.
Το IDF αφορά την μοναδικότητα ενός όρου σε ολόκληρη την συλλογή των έγγραφων

In [9]:
# Υπολογισμός συχνότητας όρων (TF)
doc_tokens = []
for document in articles:
    tokens = nltk.word_tokenize(document.lower())
    lem_tokens = [
        lemmatizer.lemmatize(token) for token in tokens if token not in stopwords and token not in string.punctuation
    ]
    doc_tokens.append(lem_tokens) 
    
tf_list = []
for tokens in doc_tokens:
    freq_dist = FreqDist(tokens)
    total_terms = len(tokens)
    tf = {term: freq / total_terms for term, freq in freq_dist.items()}
    tf_list.append(tf)

# Υπολογισμός αντίστροφης συχνότητας όρων (IDF)
total_docs = len(articles)
idf = {}
for term, doc_ids in inverted_index.items():
    idf[term] = math.log(total_docs / len(doc_ids))
    
# Υπολογισμός scores TF-IDF
scores = defaultdict(float)
for doc_id, tokens in enumerate(doc_tokens):
    tf = tf_list[doc_id]
    tf_idf = {term: tf.get(term, 0) * idf.get(term, 0) for term in tf}
    for token in query_tokens:
        scores[doc_id] += tf_idf.get(token, 0)

# Κατάταξη εγγράφων με βάση τα score
ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
print("Ranked documents:", ranked_docs)

j=1
print("Ranked Documents:")
for doc_id, score in ranked_docs:
    print(j,". ", f" {articles[doc_id][0:20]} - Score: {score:.4f}")
    j=j+1


Ranked documents: [(1, 0.030128437772162536), (0, 0.01532620530149138), (10, 0.012702800790425288), (6, 0.00982097135737358), (9, 0.0054440574816108376), (20, 0.0046535012796607485), (21, 0.0038108402371275865), (2, 0.0034793606113194494), (11, 0.0029747065142135165), (19, 0.0024867916891308764), (30, 0.0018773326132497288), (5, 0.0018181958578171592), (31, 0.0015153260480786747), (3, 0.0014551195951880362), (7, 0.0011533831392533391), (18, 0.0009989450143373763), (13, 0.0007939250494015805), (27, 0.00040188424903440413), (17, 0.0002849082416118826), (4, 0.00018967055256082955), (8, 0.0), (12, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (28, 0.0), (29, 0.0)]
Ranked Documents:
1 .   The peinirli (or pen - Score: 0.0301
2 .   Tiropita or tyropita - Score: 0.0153
3 .   Savory spinach pie i - Score: 0.0127
4 .   Pastitsio ( Greek :  - Score: 0.0098
5 .    Bougatsa ( Greek :  - Score: 0.0054
6 .   Strapatsada (Greek:  - Score: 0.0047
7 .    

<b>8ο Βήμα: Αξιολόγηση Απόδοσης</b>

Σε αυτό το βήμα αξιολογούμε την απόδοση του συτήματος ανάκτησης πληροφοριών που κατασκευάσαμε. Έτσι, δημιουργούμε ένα σύνολο ερωτημάτων, όπου για κάθε ένα ερώτημα υπολογίζουμε: 
1. Precision (Ακρίβεια), δηλαδή το ποσοστό των σχετικών εγγράφων μεταξύ των εγγράφων που ανακτήθηκαν.
2. Recall (Ανάκληση), δηλαδή το ποσοστό των σχετικών εγγράφων που ανακτήθηκαν επιτυχώς.
3. F1-Score, δηλαδή τον αρμονικό μέσο της ακρίβειας και της ανάκλησης
4. Μέση Ακρίβεια (MAP), τον μέσο όρο της ακρίβειας

In [10]:
queries = [
    "what is pizza?", 
    "food with sugar and flour", 
    "food with eggs",
    "food with cinnamon",
    "which food is sweet",
    "what food contains cheese"
]

relevant_docs = [
        [2],
        [4,8,9],
        [0,5,6,8,9,10,20,21,22,23,24,25,28,29],
        [9,15],
        [8,9,12,15,17,22,23],
        [0,5,6,9,10,18]
    ]

for q in range(6):
    query_tokens = nltk.word_tokenize(queries[q].lower())
    query_tokens = [lemmatizer.lemmatize(token) for token in query_tokens if token not in stopwords and token not in string.punctuation]

    doc_tokens = []
    for document in articles:
        tokens = nltk.word_tokenize(document.lower())
        lem_tokens = [
            lemmatizer.lemmatize(token) for token in tokens if token not in stopwords and token not in string.punctuation
        ]
        doc_tokens.append(lem_tokens) 
    
    # Υπολογισμός συχνότητας όρων (TF)
    tf_list = []
    for tokens in doc_tokens:
        freq_dist = FreqDist(tokens)
        total_terms = len(tokens)
        tf = {term: freq / total_terms for term, freq in freq_dist.items()}
        tf_list.append(tf)
    
    # Υπολογισμός αντίστροφης συχνότητας όρων (IDF)
    total_docs = len(articles)
    idf = {}
    for term, doc_ids in inverted_index.items():
        idf[term] = math.log(total_docs / len(doc_ids))
        
    # Υπολογισμός scores TF-IDF
    scores = defaultdict(float)
    for doc_id, tokens in enumerate(doc_tokens):
        tf = tf_list[doc_id]
        tf_idf = {term: tf.get(term, 0) * idf.get(term, 0) for term in tf}
        for token in query_tokens:
            scores[doc_id] += tf_idf.get(token, 0)
        
        
    print("\nsystem efficiency for the query: ", queries[q])
    
    binary_relevance = [1 if i in relevant_docs[q] and scores[i]>0.0 else 0 for i in range(len(articles))]
    true_relevance = [1 if i in relevant_docs[q] else 0 for i in range(len(articles))]
    precision = precision_score(true_relevance, binary_relevance, zero_division=0)
    recall = recall_score(true_relevance, binary_relevance, zero_division=0)
    f1 = f1_score(true_relevance, binary_relevance, zero_division=0)
    average_precision = average_precision_score(true_relevance, binary_relevance)
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Mean Average Precision (MAP): {average_precision:.4f}")





system efficiency for the query:  what is pizza?
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Mean Average Precision (MAP): 1.0000

system efficiency for the query:  food with sugar and flour
Precision: 1.0000
Recall: 0.6667
F1 Score: 0.8000
Mean Average Precision (MAP): 0.6979

system efficiency for the query:  food with eggs
Precision: 1.0000
Recall: 0.7857
F1 Score: 0.8800
Mean Average Precision (MAP): 0.8795

system efficiency for the query:  food with cinnamon
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Mean Average Precision (MAP): 1.0000

system efficiency for the query:  which food is sweet
Precision: 1.0000
Recall: 0.7143
F1 Score: 0.8333
Mean Average Precision (MAP): 0.7768

system efficiency for the query:  what food contains cheese
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Mean Average Precision (MAP): 1.0000
